### A. Librerias

In [ ]:
import os
import time
import json
import requests

import pandas as pd
from bs4 import BeautifulSoup

### B. Funciones

In [ ]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

def generar_url_google_maps(latitud, longitud):
    url = f"https://www.google.com/maps?q={latitud},{longitud}"
    return url

def generar_url_street_view(latitud, longitud):
    url = f"https://www.google.com/maps?q=&layer=c&cbll={latitud},{longitud}"
    return url

### C. Ejecución - Buscar departamentos por distritos

In [ ]:
# 1. Listar los distritos y sus urls
distritos_urls = [
                  ['surquillo','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-surquillo-lima-lima-150141']
                , ['miraflores','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-miraflores-lima-lima-150122']
                , ['barranco','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-barranco-lima-lima-150104']
                , ['breña','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-bre%C3%B1a-lima-lima-150105']
                , ['jesus_maria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-jesus-maria-lima-lima-150113']
                , ['lince', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-lince-lima-lima-150116']
                , ['magdalena', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-magdalena-del-mar-lima-lima-150120']
                , ['pueblo_libre', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-pueblo-libre-lima-lima-150121']
                , ['san_isidro', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-isidro-lima-lima-150131']
                , ['san_miguel', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['la_victoria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
            ]

In [ ]:
# 2. Extraer los datos de cada distrito
for distrito, url in distritos_urls:
    # 1. Obtener la URL por request
    response = requests.get(url)
    print(distrito, response.status_code)

    if response.status_code == 200:
        # 2. Parsear el HTML
        html = response.text

        # 3. Guardar el HTML en un archivo
        with open(os.path.join(os.getcwd(),'originals','1_inicio', distrito + '.html'), 'w') as outfile:
            outfile.write(html)

        # 4. Tiempo de espera
        time.sleep(2)

In [ ]:
# 3. Listar los proyectos y sus urls
urls_list = []
for distrito, x in distritos_urls:
    # A. Leer el html
    with open(os.path.join(os.getcwd(),'originals','1_inicio', distrito+ '.html'), 'r') as infile:
        lines = infile.readlines()

    # B. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # C. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # D. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # E. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append(base)

len(urls_list)
    

In [ ]:
# 4. Extraer los datos de cada proyecto
modelos_list = []
dataframes_list = []
for url in urls_list[:10]:
    # A. Obtener la URL por request
    response = requests.get(url)
    print(response.status_code,':' ,url)

    if response.status_code == 200:
        # Parsea el contenido HTML de la página
        soup = BeautifulSoup(response.text, 'html.parser')

        #### A. Datos generales
        # Extrae el texto de <h1>Eres Fase 2</h1>
        proyecto = soup.find('h1').get_text(strip=True)
        print(f'Proyecto: {proyecto}')

        # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
        direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
        print(f'Dirección: {direccion}')

        # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
        distrito = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
        print(f'Distrito: {distrito}')

        # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
        precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
        print(f'Precio desde: {precio_desde}')

        # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
        valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
        print(f'Valor de referencia: {valor_referencia}')

        # Listar las áreas comunes
        areas_lista = soup.find('ul', class_='Project-areas-list')
        elementos_lista = areas_lista.find_all('li')
        areas_comunes = ', '.join(elemento.get_text(strip=True) for elemento in elementos_lista)

        # Encuentra el campo de entrada para la latitud
        input_latitud = soup.find('input', id='latitude')
        latitud = input_latitud['value']
        print(f'Latitud: {latitud}')

        input_longitud = soup.find('input', id='longitude')
        longitud = input_longitud['value']
        print(f'Longitud: {longitud}')

        print('')
        print('Datos de la inmobiliaria')
        #### B. Datos de la inmobiliaria
        # Encuentra el div con la clase "bx-data-project box-st"
        div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

        if div_informacion_proyecto:
            # Encuentra todas las filas de la tabla dentro del div
            filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

            # Itera sobre las filas de la tabla e imprime la información
            for fila in filas_tabla:
                columnas = fila.find_all('td')
                if len(columnas) == 2:
                    try:
                        etiqueta = columnas[0].strong.get_text(strip=True)
                        valor = columnas[1].get_text(strip=True)
                        
                        if etiqueta == 'Tipo de inmueble':
                            tipo_inmueble = valor
                        elif etiqueta == 'Área total':
                            area_total = valor
                        elif etiqueta == 'Dormitorios':
                            dormitorios = valor
                        elif etiqueta == 'Etapa del proyecto':
                            etapa_proyecto = valor
                        elif etiqueta == 'Fecha de entrega':
                            fecha_entrega = valor
                        elif etiqueta == 'Financiamiento':
                            financiamiento = valor
                        elif etiqueta == 'Inmobiliaria Nombre':
                            inmobiliaria_nombre = valor
                        
                        print(f'{etiqueta} -> {valor}')

                    except:
                        pass

            # También puedes extraer información específica usando el método find para elementos específicos
            # Por ejemplo, para obtener la información de la inmobiliaria
            inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)
            
            print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')

        #### C. Modelos disponibles
        modelos_disponibles = soup.find_all('div', class_='Project-available-model')
        modelos_list.append([proyecto,modelos_disponibles])
        print('')
        print('Modelos disponibles')
        print(f'Cantidad de modelos: {len(modelos_disponibles)}')

        #### D. Unir los datos en un diccionario
        proyecto_data = {
            'Proyecto': proyecto,
            'Dirección': direccion,
            'Distrito': distrito,
            'Precio Desde': precio_desde,
            'Valor de Referencia': valor_referencia,
            'Latitud': latitud,
            'Longitud': longitud,
            'Tipo de Inmueble': tipo_inmueble,
            'Área Total': area_total,
            'Dormitorios': dormitorios,
            'Etapa del Proyecto': etapa_proyecto,
            'Fecha de Entrega': fecha_entrega,
            'Financiamiento': financiamiento,
            'Inmobiliaria Nombre': inmobiliaria_nombre,
            'Áreas Comunes': areas_comunes,
            'Nexo inmobiliario': url
        }

        # Convierte el diccionario en un DataFrame
        df = pd.DataFrame([proyecto_data])
        dataframes_list.append(df)

        # Tiempo de espera
        time.sleep(2)

In [ ]:
# 1. Concatenar los DataFrames
df_proyectos = pd.concat(dataframes_list)

# 2. Crear columnas de Google Maps y Street View
df_proyectos['Google Maps'] = df_proyectos.apply(lambda row: generar_url_google_maps(row['Latitud'], row['Longitud']), axis=1)
df_proyectos['Street View'] = df_proyectos.apply(lambda row: generar_url_street_view(row['Latitud'], row['Longitud']), axis=1)

# Eliminar las columnas de Latitud y Longitud
df_proyectos = df_proyectos.drop(columns=['Latitud', 'Longitud'])

df_proyectos